This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [4]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests
import json
from collections import OrderedDict


In [6]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r=requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key={API_KEY}')

In [7]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2019-11-15', 'frequency': 'daily', 'data': [['2019-11-15', 102.0, 104.1, 101.9, 104.0, None, 102061.0, 10555093.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [19]:
#Get data for 2017
data_2017 = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}')

In [20]:
#Convert data into dict
AFX2017 = data_2017.json()
#Check dict for keys
AFX2017.keys()

dict_keys(['dataset_data'])

In [21]:
#Check keys for dataset_data
AFX2017['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [22]:
#Find column names
AFX2017['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [23]:
#Convert AFX2017 into an OrderedDict for ease of use
AFX_ordered = OrderedDict()
#pull out column names and data
columns = AFX2017['dataset_data']['column_names']
data = AFX2017['dataset_data']['data']
#Create OrderedDict
for i in data:
    AFX_ordered[i[0]] = {columns[x]: i[x] for x in range(1, len(columns))}

In [24]:
#Find highest opening price
highest = max(i['Open'] for i in AFX_ordered.values() if i['Open'] != None)
highest

53.11

In [25]:
#Find lowest opening price
lowest = min(i['Open'] for i in AFX_ordered.values() if i['Open'] != None)
lowest

34.0

In [26]:
#Find the largest difference in any one day
one_day_diff = max(i['High'] - i['Low'] for i in AFX_ordered.values() if i['High'] != None and i['Low'] != None)
one_day_diff

2.8100000000000023

In [27]:
#Create a list of close prices for the entire year
close = [i['Close'] for i in AFX_ordered.values()]
#And an empty list to hold the change values
close_change = []
#loop through each day's close to append the change values to close_change
for i in range(len(close)):
    if i < len(close) -1:
        change = close[i] - close[i+1] 
        close_change.append(abs(change))
#Find the largest change in close prices
max(close_change)

2.559999999999995

In [28]:
#Find average daily trading volume - first create a list of trading volume
volume = [i['Traded Volume'] for i in AFX_ordered.values()]
#Calculate the average
avg_volume = sum(volume)/len(volume)
avg_volume

89124.33725490196

In [29]:
#Create a median function:
def find_median(lst):
    lst = sorted(lst)
    if len(lst)%2 != 0:
        middle = int((len(lst)+1)/2)
        return lst[middle]
    else:
        mid = int(len(lst)/2)
        return (lst[middle]+lst[-middle])/2
       

In [31]:
#Find median trading volume
find_median(volume)

76600.0